In [ ]:
%%time

import requests
import json

controller_url = "http://localhost:21001"
model_name = "MingAI-70B-chat-orca_v0.4_v0.2_retrained_checkpoint-2-GPTQ"
query = "아도스 연봉을 알려줘"
temperature = 0
top_p = 0
repetition_penalty = 1
max_new_tokens = 256
stop_str = ","
stop_token_ids = None


# Query worker address
ret = requests.post(controller_url + "/get_worker_address", json={"model": model_name})

worker_addr = ret.json()["address"]

# No available worker
# if worker_addr == "":
# return []

# Construct Prompt
prompt = (
    f"### System:\nThis is a system prompt, please behave and help the user.\n\n"
    f"### User: I want you to act as a google search keywords generator. I will speak to you questions. "
    f"I want you to only reply with a few search sentences based on the questions. "
    f"You must distinguish each sentence using comma(,)"
    f"do not write explanations. My first command is {query}\n\n### Assistant:"
)
# Make requests
gen_params = {
    "model": model_name,
    "prompt": prompt,
    "temperature": temperature,
    "repetition_penalty": repetition_penalty,
    "top_p": top_p,
    "max_new_tokens": max_new_tokens,
    "stop": stop_str,
    "stop_token_ids": stop_token_ids,
    "echo": False,
}

# Stream output
response = requests.post(
    worker_addr + "/worker_generate",
    headers={"User-Agent": "FastChat Client"},
    json=gen_params,
    stream=True,
    timeout=100,
)

data = json.loads(response.text)
search_keywords = data["text"].lstrip().split(",")

# return search_keywords

In [ ]:
search_keywords

In [ ]:
%%time

import requests
import json

controller_url = "http://localhost:21001"
model_name = "MingAI-70B-chat-orca_v0.4_v0.2_retrained_checkpoint-2-GPTQ"
query = "아도스 연봉을 알려줘"
temperature = 0
top_p = 0
repetition_penalty = 1
max_new_tokens = 256
stop_str = ","
stop_token_ids = None


# Query worker address
ret = requests.post(controller_url + "/get_worker_address", json={"model": model_name})

worker_addr = ret.json()["address"]

# No available worker
# if worker_addr == "":
# return []

# Construct Prompt
prompt = (
    f"### System:\nThis is a system prompt, please behave and help the user.\n\n"
    f"### User: I want you to act as a text dataset generator. "
    f"You must generate a random question, and an answer to that question."
    f"I want you to only reply with a pair of sentences. "
    f"do not write explanations.\n\n### Assistant:"
)
# Make requests
gen_params = {
    "model": model_name,
    "prompt": prompt,
    "temperature": temperature,
    "repetition_penalty": repetition_penalty,
    "top_p": top_p,
    "max_new_tokens": max_new_tokens,
    "stop": stop_str,
    "stop_token_ids": stop_token_ids,
    "echo": False,
}

# Stream output
response = requests.post(
    worker_addr + "/worker_generate",
    headers={"User-Agent": "FastChat Client"},
    json=gen_params,
    stream=True,
    timeout=100,
)

data = json.loads(response.text)
search_keywords = data["text"].lstrip().split(",")

# return search_keywords

In [ ]:
(
    f"### System:\nThis is a system prompt, please behave and help the user.\n\n"
    f"### User: I want you to act as a text dataset generator. "
    f"You must generate a random question, and an answer to that question. "
    f"I want you to only reply with a pair of sentences. "
    f"do not write explanations.\n\n### Assistant:"
)

In [ ]:
import requests
import json

query = "LK-99"
search_engine = "http://127.0.0.1:8080/"
max_urls = 5
query_url = f"{search_engine}?q={query}&format=json"

response = requests.get(query_url)
data = json.loads(response.text)

# urls = []
# # sorted_result = sorted(data['results'], key=lambda x:('google' in x['engines'], x['score']), reverse=True)
# sorted_result = sorted(data['results'], key=lambda x: x['score'], reverse=True)
# for result in sorted_result[:max_urls]:
#     if result['score'] >= 1.0:
#         urls.append(result['url'])

# return urls

In [ ]:
search_sentences = []
for search_result in data["results"][:max_urls]:
    search_sentences.append(
        (
            # f""search_result['url']
            f"Title: {search_result['title']}\n"
            f"Contents: {search_result['content']}"
        )
    )

# '\n\n'.join(search_sentences)

In [ ]:
import chromadb
from chromadb.config import Settings
import random
from fastchat.modules.embedder_adapter import Embedder, get_embedder

In [ ]:
chroma_client = chromadb.Client(Settings(anonymized_telemetry=False))
embedder = get_embedder("ddobokki/klue-roberta-base-nli-sts-ko-en")
collection = chroma_client.create_collection(
    name="context", embedding_function=embedder.embed
)

In [ ]:
from fastchat.modules.vector_store import (
    feed_data_into_collector,
    add_chunks_to_collector,
)

In [ ]:
chunk_len = 400
chunk_sep = ""
corpus = ""
# def feed_data_into_collector(collector, corpus, chunk_len=400, chunk_sep=''):
# add_chunks_to_collector(data_chunks, collector)

chunk_sep = chunk_sep.replace(r"\n", "\n")

if chunk_sep:
    data_chunks = corpus.split(chunk_sep)
    data_chunks = [
        [data_chunk[i : i + chunk_len] for i in range(0, len(data_chunk), chunk_len)]
        for data_chunk in data_chunks
    ]
    data_chunks = [x for y in data_chunks for x in y]
else:
    data_chunks = [corpus[i : i + chunk_len] for i in range(0, len(corpus), chunk_len)]


last_id = -1
new_ids = [f"id{i+last_id+1}" for i in range(len(data_chunks))]
ids += new_ids
collection.add(documents=data_chunks, ids=new_ids)

In [ ]:
result = collection.query(
    query_texts='개[1] + 작다는 뜻의 접두어 앚[2] + 명사화 접사 "이"가 합쳐져서 강아지가 됐다.',
    n_results=100,
    include=["documents", "distances"],
)

In [ ]:
from trafilatura import fetch_url, extract
from copy import deepcopy
from trafilatura.settings import DEFAULT_CONFIG

In [ ]:
DEFAULT_CONFIG["DEFAULT"]

In [ ]:
import requests

# downloaded = fetch_url("https://www.accuweather.com/ko/kr/munjeong-dong/1849778/hourly-weather-forecast/1849778", )
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}
url = "https://namu.wiki/w/%EA%B0%9C"
response = requests.get(
    url,
    headers=headers,
    timeout=5,
)
if response.status_code == 200:
    downloaded = response.content

In [ ]:
result = extract(downloaded)
print(len(result))
result

In [ ]:
query = "강아지 이름의 기원"
search_engine = "http://127.0.0.1:8080/"
max_urls = 3

query_url = f"{search_engine}?q={query}&format=json"

response = requests.get(query_url)
data = json.loads(response.text)

# urls = []
# search_sentences = []
# for search_result in data['results'][:max_urls]:
#     search_sentences.append((
#         f"{search_result['title']}\n"
#         f"{search_result['content']}"
#     ))
#     urls.append(search_result['url'])

In [ ]:
search_data = [
    {
        "idx": idx,
        "url": _data["url"],
        "title": _data["title"],
        "snippet": _data["content"],
        "raw_data": [],
    }
    for idx, _data in enumerate(data["results"][:max_urls])
]

In [ ]:
chroma_client = chromadb.Client(Settings(anonymized_telemetry=False))
embedder = get_embedder("ddobokki/klue-roberta-base-nli-sts-ko-en")
collection = chroma_client.create_collection(
    name="context", embedding_function=embedder.embed
)

chunk_len = 300
new_ids = []
data_chunks = []
for _data in search_data:
    downloaded = fetch_url(_data["url"])
    extracted = extract(downloaded)

    temp_chucks = [
        extracted[i : i + chunk_len] for i in range(0, len(extracted), chunk_len)
    ]
    data_chunks += temp_chucks
    new_ids += [f"idx{_data['idx']}_{i}" for i in range(len(temp_chucks))]

collection.add(documents=data_chunks, ids=new_ids)

In [ ]:
result = collection.query(
    query_texts=[query], n_results=5, include=["documents", "distances"]
)

In [ ]:
for ri, _idx in enumerate(result["ids"][0]):
    search_idx = int(_idx[3:].split("_")[0])
    search_data[search_idx]["raw_data"].append(result["documents"][0][ri])

In [ ]:
search_sentences = []
search_urls = []
for _data in search_data:
    if _data["raw_data"]:
        raw_data = "\n".join(_data["raw_data"])
        search_sentences.append(
            (f"{_data['title']}\n" f"{_data['snippet']}" f"{raw_data}") + "\n"
        )
        search_urls.append(_data["url"])

In [ ]:
print("\n\n".join(search_sentences))

In [ ]:
search_urls

In [ ]:
(
        f"{_data['title']}\n"
        f"{_data['snippet']}"
        f"{}"
)

In [ ]:
_data["title"]

In [ ]:
"\n".join(_data["raw_data"])